## Data Explanation For Endangered Species and World Production
---
For the Endangered Species Dataset, we have data from the IUCN Red List (Table 7) from the years 2007 to 2021 with these columns:

*   Scientific Name
*   Common Name
*   Species Type
*   IUCN Red List Category
*   Reason For Change
*   Year

The key columns are the **IUCN Red List Category** (renamed to Status), **Species Type**, and the name of the species (**scientific** or **common**). <br>

#### Explanation of the categories and reasons for change: <br>

**IUCN Red List Categories:** 
* EX - Extinct, EW - Extinct in the Wild, 
* CR - Critically Endangered 
  * CR(PE) - Critically Endangered (Possibly Extinct) 
  * CR(PEW) - Critically Endangered (Possibly Extinct in the Wild)
* EN - Endangered 
* VU - Vulnerable 
* LR/cd - Lower Risk/conservation dependent 
* NT - Near Threatened 
  * (includes LR/nt - Lower Risk/near threatened) 
* DD - Data Deficient 
* LC - Least Concern 
  * (includes LR/lc - Lower Risk, least concern).

**Reasons for change:** 
* G - Genuine status change (genuine improvement or deterioration in the species' status)
* N - Non-genuine status change 
  * (i.e., status changes due to new information, improved knowledge of the criteria, incorrect data used previously, taxonomic revision, etc.) 
* E - Previous listing was an Error. <br>
---
For the Threatened Species Dataset, we have data from the IUCN Red List (Table 5) with these columns:

*	Name
*	Mammals
*	Birds
*	Reptiles
*	Amphibians
* 	Fishes
* 	Molluscs
* 	Other Invertebrates
* 	Plants
* 	Fungi
* 	Chromista
* 	Total

All of the columns are important for analysis. <br>

---
For the World Production Dataset, we have data from 2006 to 2022 from the United Nations Industrial Development Organization with these columns:

*   Table Code
*   Table Description
* 	Country Code
*   Country Description
*   Year
*   ISIC
*   ISIC Description
*   ISIC Combination
* 	Value
*  	Table Definition Code
*	Table Description
* 	Source Code
*	Unit

The key columns are the **ISIC Description**, **Country Description**, the **Year**, and the **Value**. <br>

#### Explanation of the ISIC Description and value: <br>

**ISIC Description:** The categorization of the production of the country in the year based on ISIC code. <br>

**Value:** The value of the production of the country in the year based on ISIC code.

---

## Data Cleaning Process

We will be cleaning the data in the following ways:

*   Removing unnecessary rows, columns, and data
*   Removing duplicate data and columns
*   Renaming columns
*   Changing data types
*   Splitting columns



In [2]:
import pandas as pd
import numpy as np

## Data Cleaning: Endangered Species Dataset

In [3]:
endangered = pd.read_csv('data/endangered_species.csv')
endangered.head()


,Scientific Name,Common Name,Species Type,IUCN Red List Category,Reason For Change,Year
0,Cephalophus spadix,Abbott’s Duiker,Mammal,EN,G,2007
1,Gazella spekei,Speke’s Gazelle,Mammal,EN,G,2007
2,Gorilla gorilla,Western Gorilla,Mammal,CR,G,2007
3,Lipotes vexillifer,Baiji,Mammal,CR(PE),G,2007
4,Mazama chunyi,Dwarf Brocket Deer,Mammal,VU,N,2007


#### The following code renames the IUCN Red List Category column to Status.


In [4]:
endangered.rename(columns={'IUCN Red List Category':'Status'}, inplace=True)
endangered.head()

,Scientific Name,Common Name,Species Type,Status,Reason For Change,Year
0,Cephalophus spadix,Abbott’s Duiker,Mammal,EN,G,2007
1,Gazella spekei,Speke’s Gazelle,Mammal,EN,G,2007
2,Gorilla gorilla,Western Gorilla,Mammal,CR,G,2007
3,Lipotes vexillifer,Baiji,Mammal,CR(PE),G,2007
4,Mazama chunyi,Dwarf Brocket Deer,Mammal,VU,N,2007


#### The following code will remove the rows with missing values that are important to have.

In [5]:
# prints out the rows that have blank values in the scientific/common name column
original_data_endangered = endangered.shape[0]
print("The total amount of data originally is", original_data_endangered)
# drops the rows that have blank values in both of these columns
endangered = endangered.dropna(subset=['Scientific Name', 'Common Name'], how='all')
print("The total amount of data after removing rows with no species name is", endangered.shape[0])
# removes all rows that have blank values in the 'Year' column
endangered = endangered.dropna(subset=['Year'], how='all')
# removes all rows that have invalid species type values
species_list = list(endangered['Species Type'].unique())
# removes invalid species type values
species_list.remove("EX")
species_list.remove("VU")
species_list.remove("LR/nt")
endangered = endangered[endangered['Species Type'].isin(species_list)]
new_data_endangered = endangered.shape[0]
print("The total amount of data after removing all rows with missing important values", new_data_endangered)
print("The total amount of data removed is", original_data_endangered - new_data_endangered)

The total amount of data originally is 10013
The total amount of data after removing rows with no species name is 10003
The total amount of data after removing all rows with missing important values 9986
The total amount of data removed is 27


#### The following code makes sure that the years are in a consistent format.

In [6]:
# convert the years that have the format 2014-3 into 2014

def convert_year(year):
	'''
	Converts the year into a consistent format (for example: from 2014-3 to 2014)
	
	year (string): the year to be converted
	return (string): the converted year
	'''
	year = str(year)
	if '-' in year:
		year = year.split('-')[0]
		year = str(int(year))
		return float(year)
	elif '.' in year:
		year = year.split('.')[0]
		year = str(int(year))
		return float(year)
	elif '‐' in year:
		year = year.split('‐')[0]
		year = str(int(year))
		return float(year)
	else:
		return float(year)


print(endangered['Year'].tail())

endangered['Year'] = endangered['Year'].apply(convert_year)
endangered['Year'].tail()

10008    2021-1
10009    2021-2
10010    2021-1
10011    2021-3
10012    2021-3
Name: Year, dtype: object


10008    2021.0
10009    2021.0
10010    2021.0
10011    2021.0
10012    2021.0
Name: Year, dtype: float64

#### The following code displays where they are missing values in the Dataframe.

In [7]:
# the reason why there are null values for the scientific name and the common name because the
# species has either one or the other. if needed, we can fill in the null values with the other
endangered.isnull().any()

Scientific Name       True
Common Name           True
Species Type         False
Status                True
Reason For Change     True
Year                 False
dtype: bool

#### The following code displays the number of unique values in each important column.

In [8]:
print("Number of unique values for scientific names:", len(endangered['Scientific Name'].unique()))
print("Number of unique values for common names:", len(endangered['Common Name'].unique()))
print("Unique values for species:", endangered['Species Type'].unique())
print("Unique values for status:", endangered['Status'].unique())

Number of unique values for scientific names: 9627
Number of unique values for common names: 6091
Unique values for species: ['Mammal' 'Bird' 'Reptile' 'Fish' 'Invertebrate' 'Plant' 'Amphibian'
 'Odonata' 'Butterfly/Moth' 'Mollusc' 'Crayfish/Crab/Lobster' 'Fern'
 'Conifer' 'Cycad' 'Flowering Plants' 'Moss' 'Crustacean' 'Beetle'
 'Hymenoptera' 'Orthoptera' 'Magnolia' 'Orchid' 'Coral' 'Cricket'
 'Lepidoptera' 'Tree' 'Legume' 'Dragonfiles/Damselfiles' 'Grasshopper'
 'Horseshoe Crab' 'Bony Fish' 'Shark/Ray' 'Lamprey' 'Freshwater Shrimp'
 'Fungi' 'Mantis' 'Millipede']
Unique values for status: ['EN' 'CR' 'CR(PE)' 'VU' 'NT' 'LC' 'DD' nan 'EX' 'EW' 'LR/nt' 'En'
 'CR(PEW)']


#### The following code removes the rows that have blank values in the Status column.

In [9]:
orignal_data_endangered = endangered.shape[0]
print("The total amount of data originally is", endangered.shape[0])
endangered = endangered.dropna(subset=['Status'], how='all')
print("The total amount of data after removing rows with blank iucn red list category values is", endangered.shape[0])
print("The total amount of data removed is", orignal_data_endangered - endangered.shape[0])

The total amount of data originally is 9986
The total amount of data after removing rows with blank iucn red list category values is 9983
The total amount of data removed is 3


#### Summary Statistics

In [10]:
print("The summary statistics of the data set are:")
endangered.describe()

The summary statistics of the data set are:


,Year
count,9983.000000
mean,2016.769408
std,3.797034
min,2007.000000
25%,2014.000000
50%,2018.000000
75%,2020.000000
max,2021.000000


#### The following code groups the data by the Status and counts the number of species in each category

In [11]:
endangered.groupby('Status')['Scientific Name'].count()

Status
CR         1051
CR(PE)      117
CR(PEW)       2
DD          537
EN         2165
EW           15
EX           67
En            1
LC         2853
LR/nt         1
NT         1454
VU         1702
Name: Scientific Name, dtype: int64

#### The following code displays the number of species that are endangered per year

In [12]:
endangered_year = endangered.groupby('Year').count()
endangered_year = endangered_year.reset_index()
endangered_year = endangered_year[['Year', 'Scientific Name']]
endangered_year = endangered_year.rename(columns={'Scientific Name': 'Species Count'})
endangered_year


,Year,Species Count
0,2007.0,141
1,2008.0,212
2,2009.0,179
3,2010.0,483
4,2011.0,346
5,2012.0,412
6,2013.0,422
7,2014.0,400
8,2015.0,291
9,2016.0,754


#### The following code displays the number of species that are critically endangered per year based on Status contained within the sublist (from endangered and above)

In [13]:
endangered_sublist = ['EX', 'EW', 'CR', 'CR(PE)', 'CR(PEW)' 'EN']

endangered_critically_endangered = endangered[endangered['Status'].isin(endangered_sublist)]
endangered_critically_endangered = endangered_critically_endangered.groupby('Year').count()
endangered_critically_endangered = endangered_critically_endangered.reset_index()
endangered_critically_endangered = endangered_critically_endangered[['Year', 'Scientific Name']]
endangered_critically_endangered = endangered_critically_endangered.rename(columns={'Scientific Name': 'Species Count'})
endangered_critically_endangered

,Year,Species Count
0,2007.0,18
1,2008.0,33
2,2009.0,29
3,2010.0,64
4,2011.0,44
5,2012.0,70
6,2013.0,31
7,2014.0,69
8,2015.0,28
9,2016.0,88


#### The following code converts changes to a new CSV file

In [14]:
endangered.to_csv('data/endangered_species_new.csv', index=False)

## Data Cleaning: Threatened Species Dataset

In [15]:
threatened = pd.read_csv('data/threatened_species_by_country.csv')
threatened.head()

,Name,Mammals,Birds,Reptiles*,Amphibians,Fishes*,Molluscs*,Other Inverts*,Plants*,Fungi*,Chromists*,Total
0,Antarctica,2,5,0,0,1,0,0,0,0,0,8
1,Bouvet Island,1,1,0,0,0,0,0,0,0,0,2
2,French Southern Territories,3,14,7,0,10,0,0,2,0,0,36
3,Heard Island and McDonald Islands,1,10,0,0,2,0,0,0,0,0,13
4,South Georgia and the South Sandwich Islands,3,6,0,0,0,0,0,0,0,0,9


In [16]:
threatened.dtypes

Name              object
Mammals            int64
Birds              int64
Reptiles*          int64
Amphibians         int64
Fishes*            int64
Molluscs*          int64
Other Inverts*     int64
Plants*           object
Fungi*             int64
Chromists*         int64
Total             object
dtype: object

#### The following code renames the columns for consistency.

In [17]:
threatened.rename(columns={'Reptiles*':'Reptiles', 'Fishes*':'Fishes', "Molluscs*":"Molluscs", 
                           "Other Inverts*": "Other Inverts", "Plants*": "Plants", "Fungi*": "Fungi", "Chromists*": "Chromists"}, inplace=True)
threatened.head()

,Name,Mammals,Birds,Reptiles,Amphibians,Fishes,Molluscs,Other Inverts,Plants,Fungi,Chromists,Total
0,Antarctica,2,5,0,0,1,0,0,0,0,0,8
1,Bouvet Island,1,1,0,0,0,0,0,0,0,0,2
2,French Southern Territories,3,14,7,0,10,0,0,2,0,0,36
3,Heard Island and McDonald Islands,1,10,0,0,2,0,0,0,0,0,13
4,South Georgia and the South Sandwich Islands,3,6,0,0,0,0,0,0,0,0,9


#### The following code converts all data except for the country name to numeric

In [18]:
threatened[threatened.columns[1:]] = threatened[threatened.columns[1:]].apply(pd.to_numeric, errors='coerce')
threatened.dtypes

Name              object
Mammals            int64
Birds              int64
Reptiles           int64
Amphibians         int64
Fishes             int64
Molluscs           int64
Other Inverts      int64
Plants           float64
Fungi              int64
Chromists          int64
Total            float64
dtype: object

#### The following code displays the number of unique values in the name column.

In [19]:
print("Number of unique values for country names:", len(threatened['Name'].unique()))

Number of unique values for country names: 250


#### The following code sums the values for each species that don't have a total value.

In [20]:
threatened['Total'] = threatened[threatened.columns[1:]].sum(axis=1)

#### Summary Statistics

In [21]:
threatened.describe()

,Mammals,Birds,Reptiles,Amphibians,Fishes,Molluscs,Other Inverts,Plants,Fungi,Chromists,Total
count,250.000000,250.000000,250.000000,250.000000,250.000000,250.00000,250.000000,243.000000,250.000000,250.000000,250.000000
mean,15.220000,19.360000,12.396000,11.736000,55.244000,11.69200,43.072000,84.596708,5.048000,0.024000,435.096000
std,22.846871,22.180675,20.977689,35.829616,57.698848,31.17674,64.771353,167.357667,12.341757,0.379473,432.752782
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.000000
25%,3.000000,7.000000,2.000000,0.000000,18.500000,0.00000,3.250000,4.000000,0.000000,0.000000,166.000000
50%,8.000000,14.000000,6.000000,0.000000,43.500000,2.00000,23.500000,19.000000,0.000000,0.000000,263.000000
75%,18.000000,22.000000,10.750000,4.000000,71.750000,7.00000,55.750000,75.500000,2.000000,0.000000,579.500000
max,213.000000,161.000000,139.000000,292.000000,369.000000,320.00000,412.000000,950.000000,87.000000,6.000000,1970.000000


#### The following code converts changes to a new CSV file

In [22]:
threatened.to_csv('data/threatened_species_by_country_new.csv', index=False)

## Data Cleaning: World Production Dataset

In [23]:
world_production = pd.read_csv('data/world_production.csv')
world_production.head()


,Table Code,Table Description,Country Code,Country Description,Year,ISIC,ISIC Description,ISIC Combination,Value,Table Definition Code,Table Description.1,Source Code,Unit
0,51,Seasonally adjusted index,8,Albania,2006 Q1,10,Food products,10,65.9,51,Seasonally adjusted index,1,I
1,51,Seasonally adjusted index,8,Albania,2006 Q1,11,Beverages,11,65.9,51,Seasonally adjusted index,1,I
2,51,Seasonally adjusted index,8,Albania,2006 Q1,12,Tobacco products,12,65.9,51,Seasonally adjusted index,1,I
3,51,Seasonally adjusted index,8,Albania,2006 Q1,13,Textiles,13,58.0,51,Seasonally adjusted index,1,I
4,51,Seasonally adjusted index,8,Albania,2006 Q1,14,Wearing apparel,14,58.0,51,Seasonally adjusted index,1,I


#### The following code removes all unnecessary columns from the data set 

In [24]:
world_production = world_production[['Country Code', 'Country Description', 'Year', 'ISIC', 'ISIC Description', 'Value']]
world_production.head()

,Country Code,Country Description,Year,ISIC,ISIC Description,Value
0,8,Albania,2006 Q1,10,Food products,65.9
1,8,Albania,2006 Q1,11,Beverages,65.9
2,8,Albania,2006 Q1,12,Tobacco products,65.9
3,8,Albania,2006 Q1,13,Textiles,58.0
4,8,Albania,2006 Q1,14,Wearing apparel,58.0


#### The following code splits the year column into two columns: year and quarter

In [25]:
def split_year_quarter(year_quarter):
	'''
	Splits the year and quarter into two separate columns
	
	year_quarter (string): the year and quarter to be split
	return (list): a list containing the year and quarter
	'''
	split_value = 'Q'
	if "Y" in year_quarter:
		split_value = 'Y'
	
	year = year_quarter.split(split_value)[0].strip()
	quarter = year_quarter.split(split_value)[1].strip()
	if quarter == "":
		quarter = "Y"
	return [year, quarter]

world_production[['Year', 'Quarter']] = [split_year_quarter(yq) for yq in world_production['Year']]
world_production.head()


,Country Code,Country Description,Year,ISIC,ISIC Description,Value,Quarter
0,8,Albania,2006,10,Food products,65.9,1
1,8,Albania,2006,11,Beverages,65.9,1
2,8,Albania,2006,12,Tobacco products,65.9,1
3,8,Albania,2006,13,Textiles,58.0,1
4,8,Albania,2006,14,Wearing apparel,58.0,1


#### The following code removes all rows that contains year 2006 or 2022 and reset the indexes

In [26]:
original_data_prod = world_production.shape[0]
print("The total amount of data originally is", original_data_prod)
world_production = world_production[world_production['Year'] != 2006]
world_production = world_production[world_production['Year'] != 2022].reset_index(drop=True)

new_data_prod = world_production.shape[0]
print("The total amount of data after removing rows with the year 2006 or 2022 is", new_data_prod)
print("The total amount of data removed is", original_data_prod - new_data_prod)
world_production.head()

The total amount of data originally is 350636
The total amount of data after removing rows with the year 2006 or 2022 is 350636
The total amount of data removed is 0


,Country Code,Country Description,Year,ISIC,ISIC Description,Value,Quarter
0,8,Albania,2006,10,Food products,65.9,1
1,8,Albania,2006,11,Beverages,65.9,1
2,8,Albania,2006,12,Tobacco products,65.9,1
3,8,Albania,2006,13,Textiles,58.0,1
4,8,Albania,2006,14,Wearing apparel,58.0,1


#### Summary Statistics

In [27]:
print("The summary of the data set is:")
world_production.describe()

The summary of the data set is:


,Country Code,Value
count,350636.000000,350636.000000
mean,425.160428,112.035753
std,253.563482,920.047292
min,8.000000,-6.700000
25%,196.000000,90.100000
50%,428.000000,100.400000
75%,643.000000,112.900000
max,858.000000,318802.700000


#### The following code group by the year and ISIC description and sum the values

In [28]:
world_production_sum = world_production.groupby(['Year', 'ISIC Description']).sum()
world_production_sum = world_production_sum.reset_index()
world_production_sum = world_production_sum[['Year', 'ISIC Description', 'Value']]
world_production_sum = world_production_sum.rename(columns={'Value': 'Production Value'})
world_production_sum


C:\Users\Me\AppData\Local\Temp\ipykernel_13864\2149869270.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  world_production_sum = world_production.groupby(['Year', 'ISIC Description']).sum()


,Year,ISIC Description,Production Value
0,2005,Basic metals,53892.2
1,2005,Beverages,46945.1
2,2005,Chemicals and chemical products,44470.5
3,2005,Coke and refined petroleum products,40910.2
4,2005,"Computer, electronic and optical products",49861.2
...,...,...,...
499,2022,Tobacco products,40653.3
500,2022,Total manufacturing,77230.9
501,2022,"Water supply; sewerage, waste management",43165.3
502,2022,Wearing apparel,61779.8


#### The following code groups by ISIC description and average the values

In [29]:
world_production_avg = world_production.groupby(['ISIC Description'])
world_production_avg = world_production.groupby(['ISIC Description']).mean()
world_production_avg = world_production_avg.reset_index()
world_production_avg = world_production_avg[['ISIC Description', 'Value']]
world_production_avg = world_production_avg.rename(columns={'Value': 'Production Value'})
world_production_avg

C:\Users\Me\AppData\Local\Temp\ipykernel_13864\678825850.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  world_production_avg = world_production.groupby(['ISIC Description']).mean()


,ISIC Description,Production Value
0,Basic metals,104.626524
1,Beverages,102.112854
2,Chemicals and chemical products,103.043038
3,Coke and refined petroleum products,125.560454
4,"Computer, electronic and optical products",112.026382
5,Electrical equipment,106.219171
6,"Electricity, gas, steam & air conditioning",104.066283
7,"Fabricated metal products, except machinery",104.967973
8,Food products,101.702882
9,Furniture,105.869078


#### The following code converts changes to a new CSV file

In [30]:
world_production.to_csv('data/world_production_new.csv', index=False)